In [7]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 27.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 59.0 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import fitz  
import spacy


nlp = spacy.load('en_core_web_sm')


def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ''
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text


def extract_skills_from_text(text):
    doc = nlp(text)
    
    skills = set()
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PRODUCT']:  
            skills.add(ent.text)
    return ', '.join(skills)

resume_text = extract_text_from_pdf('/kaggle/input/shubhamsresume/Shubham Kumar (2).pdf')


extracted_skills = extract_skills_from_text(resume_text)
print(f"Extracted Skills: {extracted_skills}")


Extracted Skills: Email, API, FSD, Node.js, Next.js, KABADI TECHNO|, a Shopping Site, Sharda University, Bachelor, UI/UX Designer, JavaScript, Resume Maker, GitHub, Firebase


In [14]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv("/kaggle/input/jobposts/data job posts.csv")


df['job_info'] = df[['Title', 'JobDescription', 'JobRequirment', 'RequiredQual']].fillna('').agg(' '.join, axis=1)


resume_skills = extracted_skills  


def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text


df['cleaned_job_info'] = df['job_info'].apply(clean_text)
cleaned_resume_skills = clean_text(resume_skills)


vectorizer = CountVectorizer(stop_words='english')
job_desc_matrix = vectorizer.fit_transform(df['cleaned_job_info'])


resume_matrix = vectorizer.transform([cleaned_resume_skills])


similarity_scores = cosine_similarity(resume_matrix, job_desc_matrix)


df['similarity_score'] = similarity_scores.flatten()
recommended_jobs = df.sort_values(by='similarity_score', ascending=False)


print(recommended_jobs[['Title', 'JobDescription', 'similarity_score']].head(100))


                                       Title  \
12646                          Site Engineer   
3346            Web Site Developer/ Designer   
7326          Site Implementation Supervisor   
18825                      Node.js Developer   
7274               Email Marketing Assistant   
...                                      ...   
6602                                Designer   
14366   Head of Management and Finance Chair   
17011  Health, Safety and Environment Expert   
535                         Grants Associate   
5309                     Senior Web Designer   

                                          JobDescription  similarity_score  
12646  The Site Engineer will be responsible for mark...          0.185545  
3346   The Web Site Developer/ Designer will work und...          0.182978  
7326   The Site Implementation Supervisor is responsi...          0.175267  
18825                                                NaN          0.173344  
7274   The Email Marketing Assistant w